## I.Import library

In [1]:
import pandas as pd
import json
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)     
pd.set_option('display.max_colwidth', None) 
import re
from transformers import BertTokenizer, BertForPreTraining, BertForQuestionAnswering, BertModel, BertConfig
from transformers import XLMRobertaForQuestionAnswering, XLMRobertaTokenizer
import torch
import torch.nn as nn
from pyvi import ViTokenizer
from transformers.data.metrics.squad_metrics import compute_predictions_log_probs, compute_predictions_logits, squad_evaluate
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor, SquadProcessor, SquadExample
from transformers import AutoModel, AutoTokenizer
from transformers.data.processors.squad import squad_convert_examples_to_features
from transformers import RobertaForQuestionAnswering, RobertaConfig, WEIGHTS_NAME

c:\Users\hdang\.virtualenvs\machinelearning-lTTH8rYd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
!pip install -q ngrok


### II. Data PreProcessing

> Function Read data_origin

In [2]:
def read_data(data_dir):
    with open(data_dir, 'r', encoding = 'utf-8') as f:
        data = json.load(f)
    return data

> Execute Function

In [3]:
dataset = read_data("../data/data_origin/data_train/UIT-ViQuAD.json")

In [4]:
print(dataset['data'][1])

{'title': 'Thực vật có hoa', 'paragraphs': [{'qas': [{'question': 'Thuật ngữ thực vật hạt kín Angioosperm được định nghĩa đầu tiên trên ngôn ngữ nào?', 'answers': [{'answer_start': 67, 'text': 'tiếng Hy Lạp'}], 'id': 'uit_000042', 'is_impossible': False}, {'question': 'Angiospermae được nhà bác nào định nghĩa vào năm 1690?', 'answers': [{'answer_start': 128, 'text': 'Paul Hermann'}], 'id': 'uit_000043', 'is_impossible': False}, {'question': 'Angiospermae được nhà bác Paul Hermann định nghĩa vào năm 1690 bao gồm những loai thực nào?', 'answers': [{'answer_start': 283, 'text': 'thực vật có hoa và tạo ra các hạt được bao phủ trong các bao vỏ'}], 'id': 'uit_000044', 'is_impossible': False}, {'question': 'Cây Tuế và cây Thông thuộc loại thực vật nào?', 'answers': [{'answer_start': 985, 'text': 'thực vật hạt trần'}], 'id': 'uit_000045', 'is_impossible': False}, {'question': 'Robert Brown phát hiện sự tồn tại của noãn trần trong cây Tuế và cây Thông vào thời gian nào?', 'answers': [{'answer_s

> Tokenize and remove spaces before punctuation

In [7]:
def VnCoreNLP(text):
    # Phân tách từ
    tokens = ViTokenizer.tokenize(text).split()
    # Kết nối các từ với nhau thành chuỗi
    p = ' '.join(tokens)
    # Loại bỏ khoảng trắng không cần thiết trước các dấu câu
    p = re.sub(r'\s([.,!?;{}()*/])', r'\1', p)
    return p

> Execute Function

In [6]:
X_train_example = "Tôi đến từ Việt Nam, và tôi rất yêu đất nước này.Chúng tôi sẽ tổ chức hội thảo vào tháng sau tại TP.HCM"
x_tokenize=VnCoreNLP(X_train_example)
print(x_tokenize)

Tôi đến từ Việt_Nam, và tôi rất yêu đất_nước này. Chúng_tôi sẽ tổ_chức hội_thảo vào tháng sau tại TP. HCM


> Check Answer

In [7]:
def checkAns(answer, context_text):
  ans=context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])]
  l=ans.replace("_"," ")
  if(l == answer["text"]):
    return ans, answer["answer_start"]
  else:
    firstWord=answer["text"].split(' ')[0]
    lenWord=len(firstWord)
    for i in range(1,8):
      try:
        position_1=answer["answer_start"]-i
        word=context_text[position_1: position_1+ lenWord]
        if(firstWord == word):
          return context_text[position_1: position_1+ len(answer["text"])],position_1
        position_2=answer["answer_start"]+i
        word=context_text[position_2: position_2+ lenWord]
        if(firstWord == word):
          return context_text[position_2: position_2+ len(answer["text"])],position_2
      except:
        return ans, answer["answer_start"]
    return ans, answer["answer_start"]

> Check First_Word and Last_Word

In [8]:
def checkTempAns(answer, context_text):
  firstWord=answer["text"].split(' ')[0]
  lenWord=len(firstWord)
  for i in range(1,8):
    try:
      position_1=answer["answer_start"]-i
      word=context_text[position_1: position_1+ lenWord]
      if(firstWord == word):
        return context_text[position_1: position_1+ len(answer["text"])],position_1
      position_2=answer["answer_start"]+i
      word=context_text[position_2: position_2+ lenWord]
      if(firstWord == word):
        return context_text[position_2: position_2+ len(answer["text"])],position_2
    except:
      return context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])], answer["answer_start"]
  return context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])], answer["answer_start"]

In [9]:
from tqdm import tqdm
class VnCoreSquad(SquadV1Processor):
  def _create_examples(self, input_data, set_type):
        is_training = set_type == "train"
        examples = []
        demso=0
        for entry in tqdm(input_data):
            title = entry["title"] 
            for paragraph in entry["paragraphs"]:
                context_text = VnCoreNLP(paragraph["context"])
                for qa in paragraph["qas"]:
                    qas_id = qa["id"]
                    question_text = VnCoreNLP(qa["question"])
                    start_position_character = None
                    answer_text = None
                    answers = []

                    is_impossible = qa.get("is_impossible", False)
                    if not is_impossible:
                        if is_training:
                            answer = qa["answers"][0]
                            answer_text, start_position_character=checkAns(answer, context_text)
                            qa["answers"][0].update({"text": answer_text})
                            qa["answers"][0].update({"answer_start": start_position_character})
                        else:
                            for k in range(0, len(qa["answers"])):
                              ans, start_position=checkTempAns(qa["answers"][k],context_text)
                              qa["answers"][k].update({"text": ans})
                              qa["answers"][k].update({"answer_start": start_position})
                              
                            answers = qa["answers"]
                    example = SquadExample(
                        qas_id=qas_id,
                        question_text=question_text,
                        context_text=context_text,
                        answer_text=answer_text,
                        start_position_character=start_position_character,
                        title=title,
                        is_impossible=is_impossible,
                        answers=answers,
                    )
                    examples.append(example)
        return examples
processor = VnCoreSquad()

> Excute Function

In [10]:
train_examples = processor.get_train_examples('','../data/data_origin/data_train/train_vi.json')
dev_examples = processor.get_dev_examples('','../data/data_origin/data_train/val_vi.json')

100%|██████████| 13/13 [00:07<00:00,  1.84it/s]


In [11]:
# In ra 5 ví dụ huấn luyện đầu tiên
print("Training Examples:")
for example in train_examples[:5]:  # In ra 5 ví dụ đầu tiên
    print("Question ID:", example.qas_id)
    print("Question Text:", example.question_text)
    print("Context Text:", example.context_text[:100], "...")  # In ra 100 ký tự đầu tiên của context
    print("Answer Text:", example.answer_text)
    print("Start Position:", example.start_position)
    print("Is Impossible:", example.is_impossible)
    print("-" * 80)

Training Examples:
Question ID: uit_000001
Question Text: Tên gọi nào được Phạm Văn Đồng sử_dụng khi làm Phó_chủ_nhiệm cơ_quan Biện sự xứ tại Quế_Lâm?
Context Text: Phạm Văn_Đồng( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ_tướng đầu_tiên của nước Cộng_hòa Xã_ ...
Answer Text: Lâm_Bá_Kiệt
Start Position: 95
Is Impossible: False
--------------------------------------------------------------------------------
Question ID: uit_000002
Question Text: Phạm Văn Đồng giữ chức_vụ gì trong bộ_máy Nhà_nước Cộng_hòa Xã_hội chủ_nghĩa Việt_Nam?
Context Text: Phạm Văn_Đồng( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ_tướng đầu_tiên của nước Cộng_hòa Xã_ ...
Answer Text: Thủ_tướng
Start Position: 14
Is Impossible: False
--------------------------------------------------------------------------------
Question ID: uit_000003
Question Text: Giai_đoạn năm 1955 - 1976, Phạm Văn Đồng nắm giữ chức_vụ gì?
Context Text: Phạm Văn_Đồng( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ_tướng đầu_tiên củ

## III. Built Model

### 1.Import Model

In [2]:
phobert = AutoModel.from_pretrained("vinai/phobert-large")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")
model = RobertaForQuestionAnswering(phobert.config).from_pretrained("vinai/phobert-large")

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


> Embedding tập train

In [16]:
train_features, train_dataset = squad_convert_examples_to_features(train_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = True,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

> Embedding tập validation

In [ ]:
dev_features, dev_dataset = squad_convert_examples_to_features(dev_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

In [9]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import trange, tqdm

In [ ]:
import torch

def to_list(tensor):
    return tensor.detach().cpu().tolist()


> Evaluation

In [ ]:
device = torch.device('cuda')

In [ ]:

import os
def evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features):
    eval_sampler = SequentialSampler(dev_dataset)
    eval_dataloader = DataLoader(dev_dataset, sampler=eval_sampler, batch_size=40)
    all_results = []
#     start_time = timeit.default_timer()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
#                "token_type_ids": batch[2],
            }
            example_indices = batch[3]
            outputs = model(**inputs)
            output_model = torch.cat((outputs.start_logits, outputs.end_logits), dim=1) # concatenate start_logits and end_logits along batch dimension
        for i, example_index in enumerate(example_indices):
            eval_feature = dev_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            output = outputs.start_logits[i], outputs.end_logits[i]
            if isinstance(output, tuple):  # regular logits
                start_logits, end_logits = output
            else:  # top k logits
                start_logits, end_logits, _, _ = output.chunk(4)
            result = SquadResult(unique_id, start_logits.detach().cpu().numpy(), end_logits.detach().cpu().numpy())
            all_results.append(result)
            # all_results.append(RawResult(unique_id=feature.unique_id, start_logits=start_logits.astype(np.float64), end_logits=end_logits.astype(np.float64)))

    
    # output_prediction_file = os.path.join("./", "predictions_{}.json".format(""))
    # output_nbest_file = os.path.join("./", "nbest_predictions_{}.json".format(""))
    # output_null_log_odds_file = os.path.join("./", "null_odds_{}.json".format(""))
    predictions = compute_predictions_logits(
            dev_examples,
            dev_features,
            all_results,
            20,
            128,
            False,
          None,
          None,
          None,
            True,
            False,
            0.0,
            tokenizer,
        )
    results = squad_evaluate(dev_examples, predictions)
    return results

> Fine-tuning

In [ ]:
num_epochs = 4
tb_writer = SummaryWriter()
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=16)
t_total = len(train_dataloader) // 1 * num_epochs

output_dir = os.path.join('/content/drive/MyDrive/pho-bert/', 'final_model')
no_decay = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=814, num_training_steps=t_total
)

device = torch.device('cuda')

model.to(device)

global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0

model.zero_grad()
train_iterator = trange(
    epochs_trained, int(num_epochs), desc="Epoch", disable=-1 not in [-1, 0]
)

from functools import partial
tqdm = partial(tqdm, position=0, leave=True)
max_f1 = 0
patient = 0
for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
    for step, batch in enumerate(epoch_iterator):
        model.train()
        batch = tuple(t.to(device) for t in batch)
        
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        global_step += 1

        if global_step % 5000 == 0:
            print(" global_step = %s, average loss = %s", global_step, tr_loss/global_step)
    model.eval()
    model_2 = model
    tokenizer_2 = tokenizer
    results = evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features)
    print("exact: " + str(results['exact']))
    print(results['f1'])
    if results['f1'] >= max_f1:
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(output_dir)
        max_f1 = results['f1']
        patient = 0
    else:
        patient = patient+1
    if patient == 3:
        break

del model

In [10]:
device = torch.device('cpu')

In [11]:
model_2 = RobertaForQuestionAnswering(phobert.config).from_pretrained("../final_model").to(device)
tokenizer_2 = AutoTokenizer.from_pretrained("vinai/phobert-large")

> PreProcessing Data

In [19]:
import string
def preprocess_text(text):
    # Tách từ
    text= VnCoreNLP( text)
    # Tạo bảng dịch loại bỏ tất cả dấu câu ngoại trừ "-" và "/"
    exclude_chars = string.punctuation.replace('-', '').replace('/', '').replace('_','').replace('.','')
    translator = str.maketrans('', '', exclude_chars)
    text_processed = text.translate(translator)
    return text_processed


In [25]:
def split_text(text, max_length=300):
    text=VnCoreNLP(text)
    words = text.split()  # Tách các từ trong đoạn văn thành danh sách
    segments = []
    current_segment = ""
    for word in words:
        temp=len(current_segment) + len(word)+1
        if temp<= max_length:  # Kiểm tra độ dài của đoạn văn bản
            current_segment += word + ' '  # Thêm từ vào đoạn văn bản hiện tại
        else:
            # Thêm đoạn văn bản vào danh sách các đoạn
            segments.append(current_segment.strip())
            print(len(current_segment))
            current_segment = word + ' '  # Bắt đầu một đoạn mới
    if current_segment:
        # Thêm đoạn văn bản cuối cùng vào danh sách các đoạn
        segments.append(current_segment.strip())
    return segments
def answer_question(contexts, question):
    answers=[]
    for context in contexts:
        # Tokenize inputs
        inputs = tokenizer_2.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
        
        # Move inputs tensor to device
        inputs = {key: value.to(torch.device('cpu')) for key, value in inputs.items()}
        
        # Get start and end logits for the answer
        with torch.no_grad():
            start_logits, end_logits = model_2(**inputs, return_dict=False)
        
        # Find the answer
        start_idx = torch.argmax(start_logits, dim=-1)  
        end_idx = torch.argmax(end_logits, dim=-1) + 1
  
        
        # Convert tokens to string
        answer_tokens = tokenizer_2.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx])
        answer = tokenizer_2.convert_tokens_to_string(answer_tokens).strip()
        answers.append(answer)
    return answers

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def select_best_answer(answers):
    answers = [ans for ans in answers if ans.strip()]
    if not answers:
        return ""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(answers)
    tfidf_scores_per_answer = np.sum(tfidf_matrix, axis=1)
    best_answer_index = np.argmax(tfidf_scores_per_answer)
    best_answer = answers[best_answer_index]
    return best_answer


In [28]:
# Define context and question
context = """Bộ Thông tin và Truyền thông trả lời vấn đề này như sau:Tại Điều 7 Thông tư số 24/2014/TT-BTTTT ngày 29/12/2014 của Bộ Thông tin và Truyền thông quy định chi tiết về hoạt động quản lý, cung cấp và sử dụng dịch vụ trò chơi điện tử trên mạng quy định về vật phẩm ảo, đơn vị ảo, điểm thưởng, như sau:- Doanh nghiệp cung cấp dịch vụ trò chơi điện tử chỉ được khởi tạo các vật phẩm ảo, đơn vị ảo, điểm thưởng trong trò chơi điện tử theo đúng nội dung, kịch bản mà doanh nghiệp đã báo cáo trong hồ sơ được cấp quyết định phê duyệt nội dung, kịch bản trò chơi điện tử và trong báo cáo định kỳ của doanh nghiệp.- Người chơi được dùng điểm thưởng hoặc đơn vị ảo có trong tài khoản trò chơi điện tử của mình để đổi lấy vật phẩm ảo do doanh nghiệp cung cấp dịch vụ trò chơi điện tử khởi tạo.- Doanh nghiệp cung cấp dịch vụ trò chơi điện tử có nghĩa vụ quản lý vật phẩm ảo, đơn vị ảo, điểm thưởng trong trò chơi điện tử theo đúng quy tắc trò chơi đã công bố và phù hợp với nội dung, kịch bản trò chơi đã được phê duyệt.- Vật phẩm ảo, đơn vị ảo, điểm thưởng chỉ được sử dụng trong phạm vi trò chơi điện tử và theo đúng mục đích mà doanh nghiệp đã báo cáo. Vật phẩm ảo, đơn vị ảo, điểm thưởng không phải là tài sản, không có giá trị quy đổi ngược lại thành tiền, thẻ thanh toán, phiếu thưởng hoặc các hiện vật có giá trị giao dịch bên ngoài trò chơi điện tử.- Không mua, bán vật phẩm ảo, đơn vị ảo, điểm thưởng giữa những người chơi với nhau.Như vậy, việc các doanh nghiệp cung cấp dịch vụ trò chơi điện tử trên mạng khởi tạo các vật phẩm ảo, đơn vị ảo, điểm thưởng trong các trò chơi đã được cấp phép phát hành để cung cấp cho người chơi theo đúng nội dung, kịch bản mà doanh nghiệp đã báo cáo trong hồ sơ được cấp quyết định phê duyệt nội dung, kịch bản trò chơi điện tử và trong báo cáo định kỳ của doanh nghiệp là phù hợp quy định của pháp luật hiện hành.Tuy nhiên, khi cung cấp dịch vụ trò chơi điện tử trên mạng, các doanh nghiệp cung cấp dịch vụ trò chơi điện tử trên mạng và người chơi (người sử dụng dịch vụ) phải tuân thủ đầy đủ các quy định tại Điều 7 Thông tư số 24/2014/TT-BTTTT, các quy định tại Nghị định số 72/2013/NĐ-CP, Nghị định số 27/2018/NĐ-CP và các quy định của pháp luật hiện hành có liên quan. Trường hợp nếu vi phạm sẽ bị xử lý theo quy định tại Điểm a, Khoản 6, Điều 104 Nghị định số 15/2020/NĐ-CP ngày 3/2/2020 của Chính phủ về xử phạt vi phạm hành chính trong lĩnh vực bưu chính, viễn thông, tần số vô tuyến điện, công nghệ thông tin và giao dịch điện tử (sử dụng vật phẩm ảo, đơn vị ảo, điểm thưởng ngoài phạm vi trò chơi điện tử và không theo đúng mục đích mà doanh nghiệp đã báo cáo; quy đổi ngược lại thành tiền, thẻ thanh toán, phiếu thưởng hoặc các hiện vật có giá trị giao dịch bên ngoài trò chơi điện tử; mua, bán vật phẩm ảo, đơn vị ảo, điểm thưởng giữa những người chơi với nhau)."
11,Nguyễn Đình Phúc Lộc,12:28 01/10/2022,Quy định về ghi quê quán trên giấy khai sinh,"Ngày 20/9/2022, tôi đến UBND phường Tân Thới Hoà, quận Tân Phú, TPHCM làm thủ tục đăng ký khai sinh cho con. Tôi quê quán ở Bình Định nhưng nơi sinh ở tại TPHCM, nên theo tôi, trong phần quê quán của con tôi tại bản khai giấy khai sinh phải được ghi là TPHCM. Tuy nhiên cán bộ hộ tịch hướng dẫn tôi phải ghi phần quê quán trên giấy khai sinh của con à Bình Định (theo như phần quê quán trên giấy căn cước của tôi) mới tiếp nhận hồ sơ.  Xin hỏi, việc ghi quê quán trên bản khai giấy đăng ký khai sinh của con tôi thể hiện thế nào là đúng? Việc từ chối tiếp nhận hồ sơ đăng ký khai sinh với lý do phần quê quán không ghi theo hướng dẫn là đúng quy định không?","UBND phường Tân Thới Hòa, quận Tân Phú, TPHCM trả lời vấn đề này như sau:Ngày  20/9/2022, bộ phận tiếp nhận và hoàn trả hồ sơ của UBND phường Tân Thới  Hòa, quận Tân Phú, TPHCM tiếp nhận hồ sơ yêu cầu đăng ký khai sinh của  con ông Nguyễn Đình Phúc Lộc là trẻ Nguyễn Phúc Uyên Hòa (sinh ngày  10/9/2022).Trên tờ khai đăng ký khai sinh đề nghị nội dung quê  quán của Nguyễn Phúc Uyên Hòa theo quê quán của cha là TPHCM. Qua kiểm  tra giấy tờ, hồ sơ do ông Lộc cung cấp thì cha của trẻ Nguyễn Phúc Uyên  Hòa là ông Nguyễn Đình Phúc Lộc có quê quán Nhơn Mỹ, thị xã An Nhơn,  Bình Định và mẹ Nguyễn Thị Hoàn, có quê quán Trường Thọ, thành phố Thủ  Đức, TPHCM.Qua trình bày của ông Lộc thì quê quán của trẻ là theo  nơi sinh của cha là TPHCM. Cán bộ tiếp nhận hồ sơ đã giải thích căn cứ  theo quy định tại Khoản 8 Điều 4Luật Hộ tịch(có hiệu lực ngày 1/1/2016), tại điểm d khoản 1 Điều 4 Nghị định123/2015/NĐ-CPngày 15/11/2015 của Chính phủ quy định chi tiết một số điều và biện  pháp thi hành Luật Hộ tịch thì: ""Quê quán của người được đăng ký khai  sinh được xác định theo quê quán của cha hoặc mẹ, theo thỏa thuận của  cha, mẹ hoặc theo tập quán được ghi trong tờ khai khi đăng ký khai  sinh"". Nên trẻ Nguyễn Phúc Uyên Hòa chọn quê quán theo cha là quê quán  Nhơn Mỹ, thị xã An Nhơn, Bình Định.Cán bộ tiếp nhận hồ sơ hướng  dẫn phần ghi quê quán căn cứ theo quy định tại khoản 8 Điều 4 Luật Hộ  tịch (có hiệu lực ngày 1/1/2016). Tại điểm đ khoản 1 Điều 4 Nghị định  123/2015/NĐ-CP quy định, tiếp nhận hồ sơ và thực hiện việc đăng ký khai  sinh của trẻ theo quy định.Ngày 23/9/2022, UBND phường Tân Thới  Hòa đã liên hệ ông Lộc để trao đổi, ghi nhận lại sự việc phản ánh kiến  nghị trên. Qua trao đổi, ông Lộc thống nhất việc ghi vào khai sinh của  trẻ Nguyễn Phúc Uyên Hòa, phần quê quán Nhơn Mỹ, thị xã An Nhơn, Bình  Định. Sau buổi trao đổi, UBND phường Tân Thới Hòa đã trả kết quả khai  sinh gồm bản chính và bản trích lục khai sinh cho ông Lộc."""


question = "Ngày 20/9/2022, tôi đến UBND phường Tân Thới Hoà, quận Tân Phú, TPHCM làm thủ tục đăng ký khai sinh cho con. Tôi quê quán ở Bình Định nhưng nơi sinh ở tại TPHCM, nên theo tôi, trong phần quê quán của con tôi tại bản khai giấy khai sinh phải được ghi là TPHCM. Tuy nhiên cán bộ hộ tịch hướng dẫn tôi phải ghi phần quê quán trên giấy khai sinh của con à Bình Định (theo như phần quê quán trên giấy căn cước của tôi) mới tiếp nhận hồ sơ.  Xin hỏi, việc ghi quê quán trên bản khai giấy đăng ký khai sinh của con tôi thể hiện thế nào là đúng? Việc từ chối tiếp nhận hồ sơ đăng ký khai sinh với lý do phần quê quán không ghi theo hướng dẫn là đúng quy định không?"


# Chia văn bản thành các đoạn nhỏ
segments = split_text(context)

print(segments)
# Lấy các câu trả lời từ các đoạn
answers = answer_question(segments,question)

print("Câu trả lời:",answers)

297
299
294
295
298
300
298
295
294
299
298
296
300
294
294
292
298
299
['Bộ Thông_tin và Truyền_thông trả_lời vấn_đề này như sau : Tại Điều 7 Thông_tư số 24/ 2014/ TT - BTTTT ngày 29/ 12/ 2014 của Bộ Thông_tin và Truyền_thông quy_định chi_tiết về hoạt_động quản_lý, cung_cấp và sử_dụng dịch_vụ trò_chơi điện_tử trên mạng quy_định về vật_phẩm ảo, đơn_vị ảo, điểm thưởng,', 'như sau : - Doanh_nghiệp cung_cấp dịch_vụ trò_chơi điện_tử chỉ được khởi tạo các vật_phẩm ảo, đơn_vị ảo, điểm thưởng trong trò_chơi điện_tử theo đúng nội_dung, kịch_bản mà doanh_nghiệp đã báo_cáo trong hồ_sơ được cấp quyết_định phê_duyệt nội_dung, kịch_bản trò_chơi điện_tử và trong báo_cáo định_kỳ', 'của doanh_nghiệp. - Người chơi được dùng điểm thưởng hoặc đơn_vị ảo có trong tài_khoản trò_chơi điện_tử của mình để đổi lấy vật_phẩm ảo do doanh_nghiệp cung_cấp dịch_vụ trò_chơi điện_tử khởi tạo. - Doanh_nghiệp cung_cấp dịch_vụ trò_chơi điện_tử có nghĩa_vụ quản_lý vật_phẩm ảo, đơn_vị ảo, điểm', 'thưởng trong trò_chơi điện_

> split the dataset into file train/val/test

In [8]:
split_ratio = 0.8  
with open('../data/data_origin/data_train/UIT-ViQuAD.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)['data']

# Split the dataset into training and validation sets
split_index = int(len(dataset) * split_ratio)
split_10 = int(len(dataset) * 0.1)
train_data = dataset[:split_index]
remain = dataset[split_index:]
val = remain[:split_10]
test = remain[split_10:]
# Save the training set to a new file
with open('../data/data_origin/data_train/train_vi.json', 'w', encoding='utf-8') as f:
    json.dump({'data': train_data}, f, ensure_ascii=False)

# Save the validation set to a new file
with open('../data/data_origin/data_train/val_vi.json', 'w', encoding='utf-8') as f:
    json.dump({'data': val}, f, ensure_ascii=False)

    # Save the validation set to a new file
with open('../data/data_origin/data_train/test_vi.json', 'w', encoding='utf-8') as f:
    json.dump({'data': test}, f, ensure_ascii=False)